In [15]:
import numpy as np
import gradio as gr
from PIL import Image
import torch

In [16]:
# tracking_uri=r"C:\Users\User\Downloads\60 days of python\day-36(Experiment tracking using MLFlow\mlruns\347303341436223307\models\m-2b6b8ec431cb478fb7719c0601c34041\artifacts\data\model.pth"
# run_id='b7275be8f0ca405f9a38248c21d85631'
# experiment_id='347303341436223307'
import mlflow.pytorch

model_uri = r"C:\Users\User\Downloads\60 days of python\day-36(Experiment tracking using MLFlow\mlruns\347303341436223307\models\m-8bc2e829cbc849b6a7e406fc8c1367ab\artifacts\data\model.pth"
# model_uri = r"C:\\Users\\User\\Downloads\\60 days of python\\day-36(Experiment tracking using MLFlow\\mlruns\\347303341436223307\\models\\m-2b6b8ec431cb478fb7719c0601c34041\\artifacts\\data\\model.pth"



In [17]:
print(model_uri)

C:\Users\User\Downloads\60 days of python\day-36(Experiment tracking using MLFlow\mlruns\347303341436223307\models\m-8bc2e829cbc849b6a7e406fc8c1367ab\artifacts\data\model.pth


In [18]:

import torch

model = torch.load(model_uri, weights_only=False)



In [19]:
def preprocess_image(image):
    image = image.convert("L")#convert into grayscale image .for rgb it will be "RGB"
    image=image.resize((28,28))
    image=np.array(image)/255 #converts each pixel between 0 and 1
    image=((image-0.1307)/0.3081) #standardization
    image=torch.tensor(image).unsqueeze(0).float()#convert to tensor and added a batch dimension  i.e. dimension 1,1,28,28
    return image

def predict_image(image_path):
    image=Image.open(image_path)#1(batch number),1(channel number(grayscale)),28,28
    image=preprocess_image(image)
    model.eval()
    with torch.no_grad():
        output=model(image)#output shape:(1,10) i.e.[0.2,.1,,,,,]
        prediction=output.argmax(dim=1).item()
    return prediction


In [20]:
""" Design interface for the App"""
gr_interface = gr.Interface(
    fn=predict_image,# interface ta j input nibe seta kon function k provide korbe.
    inputs=gr.components.Image( #gr.components.Image :This tells Gradio that the input will be an image. The user will see an image upload box in the web interface.
        type='filepath'#'filepath' means: Gradio will save the uploaded image to a temporary file on disk, and your function will receive the path to that file as a string.
    ),
    outputs=gr.components.Label(),
    title="Hand written digit recognition app",
    description="Upload a grayscal image to predict handwritten digit.",
)

In [21]:
gr_interface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


In [23]:
import time
import gradio as gr

def slow_echo(message, history):
    for i in range(len(message)):
        time.sleep(0.05)
        yield "You typed: " + message[: i + 1]

demo = gr.ChatInterface(
    slow_echo,
    #"messages" :Your function receives full chat history and latest message.
    #"markdown" :Your function receives only the latest text (simple text chat).
    flagging_mode="manual",
    flagging_options=["Like", "Spam", "Inappropriate", "Other"],
    save_history=True,
)

if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [24]:
import gradio as gr
from transformers import pipeline

qa_pipeline = pipeline(
    "question-answering",
    model="distilbert-base-uncased-distilled-squad"
)



Device set to use cpu


In [33]:
import gradio as gr
import time
from transformers import pipeline

def question_answering(history, context):
    """
    New Gradio 4+ signature:
    - history: list of [user, bot] pairs
    - context: the additional textbox input
    """
    if not history:
        return [], "No question asked yet."

    question = history[-1][0]  # latest user message

    try:
        result = qa_pipeline(context=context, question=question)
        answer = result['answer']
        confidence = result['score']

        full_response = f"**Answer:** {answer}\n\n**Confidence:** {confidence:.2%}"

        # Update history
        history[-1][1] = ""  # start streaming
        for i in range(len(full_response)):
            time.sleep(0.02)
            history[-1][1] = full_response[:i+1]
            yield history, ""

        yield history, ""

    except Exception as e:
        error_msg = f"Error: {str(e)}"
        history[-1][1] = error_msg
        yield history, ""

# Create the interface
demo = gr.ChatInterface(
    fn=question_answering,
    additional_inputs=[
        gr.Textbox(label="Context", placeholder="Enter context here", lines=7)
    ],
    title="Question Answering Bot",
    description="Provide a context and ask questions about it.",
)

demo.launch()

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\gradio\utils.py:1185: UserWarning: Expected 2 arguments for function <function question_answering at 0x0000023187C9CE00>, received 3.
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\gradio\utils.py:1193: UserWarning: Expected maximum 2 arguments for function <function question_answering at 0x0000023187C9CE00>, received 3.
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\gradio\utils.py:1185: UserWarning: Expected 2 arguments for function <function question_answering at 0x0000023187C9CEA0>, received 3.
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\s

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [26]:
hist=question_answering(
    context="The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower. Constructed from 1887 to 1889, it was initially criticized by some of France's leading artists and intellectuals for its design.",
    question="Who designed the Eiffel Tower?",
    history=None,

)

In [27]:
print(hist)

<generator object question_answering at 0x0000023185692560>
